In [37]:
import numpy as np
import pandas as pd
import pymongo
import csv

In [17]:
from oaipmh.client import Client
from oaipmh.metadata import MetadataRegistry, oai_dc_reader
from datetime import datetime

In [3]:
URL = "http://export.arxiv.org/oai2"

In [4]:
registry = MetadataRegistry()
registry.registerReader("oai_dc", oai_dc_reader)
client = Client(URL, registry)

In [49]:
%%time

cnt = 0
articles_chunk = []
csvfile, csvwriter = None, None
it = iter(client.listRecords(metadataPrefix="oai_dc"))

while True:
    try:
        header, metadata, _ = next(it)
        article = metadata.getMap()
        articles_chunk.append(article)
        if cnt % 1000 == 0:
            if cnt == 0:
                csvfile = open("arxiv.csv", "w")
            else:
                csvfile.close()
                csvfile = open("arxiv.csv", "a")
            csvwriter = csv.DictWriter(csvfile, fieldnames=article.keys())
            if cnt == 0:
                csvwriter.writeheader()
            csvwriter.writerows(articles_chunk)
            csvfile.close()
            articles_chunk = []
        cnt += 1
        if cnt % 10000 == 0:
            print(cnt, datetime.now())
    except StopIteration:
        break
    except BaseException:
        pass

if len(articles_chunk):
    with open("arxiv.csv", "a") as csvfile:
        csvwriter = csv.DictWriter(csvfile, fieldnames=articles_chunk[-1].keys())
        csvwriter.writerows(articles_chunk)
    cnt += len(articles_chunk)
    print(cnt, datetime.now())
    articles_chunk = []

10000 2017-09-06 13:30:46.094659
20000 2017-09-06 13:34:56.428939
30000 2017-09-06 13:39:25.584351
40000 2017-09-06 13:43:24.101449
50000 2017-09-06 13:47:33.368394
60000 2017-09-06 13:51:51.025187
70000 2017-09-06 13:55:56.454927
80000 2017-09-06 14:00:11.937482
90000 2017-09-06 14:04:28.512952
100000 2017-09-06 14:08:47.447607
110000 2017-09-06 14:13:15.475920
120000 2017-09-06 14:17:16.738396
130000 2017-09-06 14:21:28.571796
140000 2017-09-06 14:25:38.839243
150000 2017-09-06 14:29:40.694931
160000 2017-09-06 14:33:53.919737
170000 2017-09-06 14:38:13.391487
180000 2017-09-06 14:42:23.746478
190000 2017-09-06 14:46:25.346377
200000 2017-09-06 14:50:27.865204
210000 2017-09-06 14:54:25.065871
220000 2017-09-06 14:58:27.210410
230000 2017-09-06 15:02:41.060281
240000 2017-09-06 15:07:03.129660
250000 2017-09-06 15:11:06.433347
260000 2017-09-06 15:15:15.640879
270000 2017-09-06 15:19:15.688176
280000 2017-09-06 15:23:25.309146
290000 2017-09-06 15:27:41.483744
300000 2017-09-06 15:32

---